In [5]:
### Import all extra functions ###

import plotly.graph_objects as go
import numpy as numpy
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate
import dash_bootstrap_components as dbc
import time
import Utils ### Custom function scripts, Thanks to Tom Irons of the Teale group (UoN School of Chemistry) for molecule generation code. 19/7/22 ###
from IPython.display import display, HTML
import json
display(HTML("<style>.container { width:100% !important; }</style>"))


# Variables input section

In [6]:
### Variables ###
work_dir = "/data/COLABS/ElecField/ExoDielsAlder/" ### Folder with all relevant files. Needs to end in / ###
server = "loginchem01.nottingham.ac.uk" ### Set to login001.augusta.nottingham.ac.uk for HPC access, if "LOCAL", no server will be used. ###
qm_package = "QChem" ### Set to either QChem or GAUSSIAN ###
host_work_dir = "/home/pcyra2/COLABS/ElecField/Exo/" ### Folder on server for calculations. Needs to end in / ###
start_coordinate = "exoDA.xyz" ### Currently only supports .xyz files, although new file types can be added easily ###
calculate_data = False ### Whether to run the data calculations (if you're just doing analysis, set to False) ###
qm_functional = "PBE"
qm_basis_set = "6-31+G*"
qm_dispersion = "EMPIRICAL_GRIMME"
system_spin = "1" ### Spin/multiplicity in the format the QM package requires, no calculation is performed on this and is used "as is" ###
system_charge = "0"
spherical_datapoints = 300 ### Number of single point calculations required ###
charge_value = 3 ### Value of point charges around the sphere ###
radius = 30 ### Radius of sphere ###
threads = 4 ### Number of threads to give each single point calculation ###
mem = 2 ### Ammount of memory to give each single point calculation ###
random = True ### Decides whether spherical points are random or uniform ###
gen_newcoord = True ### Decides whether to generate new coordinate file in format for later visualisation ###
gen_newcharge = False
recenter = False ### Decides whether to recenter the coordinates if not already in the center. ###
center_type = "simple" ### either simple, mass weighted, atom, or translate ###
center_info = [] ### if atom, put atom number in array, if translate, put coordinates in array ([x,y,z]) ###
coord_filename = "coords_formatted.txt" ### Coord file, with format X, Y, Z, Atomic number (hint: H = 1, C = 6, N = 7, O = 8), (only change if adding own file) ###
energies_filename = "Energy.xyzc" ### Spherical energy file, with format X, Y, Z, C (only change if you're adding your own file) ###
resolution=500 ### Resolution of sphere, increase to improve contours but decrease for speed, Suggest 100 ###


 # Data generation section

In [7]:
### Calculation setup ###
if calculate_data == True:
    (type_atom, x_atom, y_atom, z_atom) = Utils.CoordGet(work_dir,start_coordinate) ### loads in initial coordinates ###

    (x_atom, y_atom, z_atom) = Utils.CoordCheck(type_atom, x_atom, y_atom, z_atom, recenter, center_type, center_info) ### Checks the coordinates to make sure they exist and locates the center of the system ###

    if gen_newcoord == True:    Utils.FormatCoord(type_atom, x_atom, y_atom, z_atom,work_dir) ### Generates formatted coordinate file if requested ###

    if gen_newcharge == True:    
        if random == True: Utils.RandomCoords(work_dir,spherical_datapoints,radius) ### Generates random spherical points ###
        else: ### Generates non-random points about the sphere ###
            print("Warning, this method is currently unsupported and probably doesnt work. DONT USE UNLESS YOU HAVE FIXED IT")
            (px,py,pz) = Utils.SphereGen(0,0,0,radius,spherical_datapoints)
            with open(work_dir + "ChargeP.xyz",'w') as f:
                for i in range(len(px)):
                    print(str(px[i])+"\t"+str(py[i])+"\t"+str(pz[i]),file=f)
            with open(work_dir + "ChargeN.xyz",'w') as f:
                for i in range(len(px)):
                    print(str(-px[i])+"\t"+str(-py[i])+"\t"+str(-pz[i]),file=f)

    print("Generating input files")

    Utils.GenSP(type_atom, x_atom, y_atom, z_atom, work_dir, charge_value, mem, threads, qm_functional, qm_basis_set, system_charge, system_spin, qm_dispersion, qm_package)

    Utils.SPRun(work_dir,mem,threads,spherical_datapoints,host_work_dir,server,qm_package)
else:
    print("Data should already be calculated.")


Data should already be calculated.


In [8]:
### For local runs, this is required. ###

failed = Utils.OutputExtract(work_dir,spherical_datapoints,threads,qm_package)

Total number of failed jobs is 0


# Analysis section

In [9]:
### THIS CELL IS DEPRECIATED, Kept for debugging perposes only ###


# ### Variables, This is the only part that you should edit. ###
# # work_dir = "/data/COLABS/Christof_MARKO/planarB3LYP/" ### Folder with all relevant files ###
# coord_filename = "coords_formatted.txt" ### Coord file, with format X, Y, Z, Atomic number (hint: H = 1, C = 6, N = 7, O = 8)###
# energies_filename = "Energy.xyzc" ### Spherical energy file, with format X, Y, Z, C ###
# # radius = 9 ### Radius of sphere ###
# resolution=20 ### Resolution of sphere ###

In [10]:
start_time = time.perf_counter()
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

### Extract energy data ###
(x_data, y_data, z_data, en_data) =  Utils.EnergyExtract(work_dir,energies_filename)
MolData = numpy.genfromtxt(work_dir + coord_filename)

data_loaded = time.perf_counter()

print(f"Time taken to import data is {data_loaded - start_time} seconds")

### Generate spheres ###
(x_sphere, y_sphere, z_sphere) = Utils.SphereGen(0,0,0,1,resolution)

colour_start = time.perf_counter()

print(f"Time taken to generate sphere is {colour_start - data_loaded} seconds")

### Generate colors and contours ###
(col_sphere, x_contour, y_contour, z_contour, val_contour) = Utils.GenColorsFaster(x_sphere, y_sphere, z_sphere, x_data, y_data, z_data, en_data, radius)

color_end = time.perf_counter()

print(f"Time taken to interpolate and generate color data is {color_end - colour_start} seconds")


Time taken to import data is 0.006896245060488582 seconds
Time taken to generate sphere is 0.04173312499187887 seconds


Color progress:   0%|          | 0/1000 [00:00<?, ?it/s]

Step 0 progress:   0%|          | 0/500 [00:00<?, ?it/s]

Step 1 progress:   0%|          | 0/500 [00:00<?, ?it/s]

Step 2 progress:   0%|          | 0/500 [00:00<?, ?it/s]

...

Step 996 progress:   0%|          | 0/500 [00:00<?, ?it/s]

Step 997 progress:   0%|          | 0/500 [00:00<?, ?it/s]

Step 998 progress:   0%|          | 0/500 [00:00<?, ?it/s]

Step 999 progress:   0%|          | 0/500 [00:00<?, ?it/s]

Coloring done
Time taken to interpolate and generate color data is 66.47847786196508 seconds


__[Link to Visualisation](http://127.0.0.1:8050/)__

In [ ]:

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = html.Div([
    html.H1(children="Spherical visualisation tool",style={'text-align' : 'center'},className="bg-primary text-white m-1 p-3 text-decoration-underline"),
    html.Div([
        html.H3("Sphere opacity"),
        dcc.Slider(id='alphval', min=0, max=1, step=0.05, value=0.8),
    ], className="border border-3 m-2"),
    html.Div([
        html.Div([
            html.H3("Interpolation data"),
            dcc.RadioItems(id='interp',options=["None","Sphere","Contours"],value="Sphere",inline=True),
        ],className="vstack gap-2 m-2"),
        html.Div([
            html.H3("Molecule texture style"),
            dcc.RadioItems(id='texture',options=["matte","shiny","orbs"],value="orbs",inline=True),
        ],className="vstack gap-2 m-2"),
        html.Div([
            html.H3("Molecule draw style"),
            dcc.RadioItems(id='draw_type',options=["ball_and_stick","tubes","wireframe","spacefilling"],value="tubes",inline=True),
        ],className="vstack gap-2 m-2"),
    ],className="hstack gap-3 bg-secondary m-2 text-black border border-3"),
    html.Div([dcc.Graph(id="graph", style={'text-align' : 'center'}),], className="gap-3 bg-secondary m-2 text-black border border-3 vstack")
])


@app.callback(Output('graph','figure'),Input('alphval','value'),Input('interp','value'),Input('texture','value'),Input('draw_type','value'))
def update_graph(alphval,interp,texture,draw_type):
    Fig = go.Figure(layout = go.Layout(title="Visualisation",uirevision='camera'))
    Fig.update_layout(autosize=True)#, width=1000, height=1000)
    Molecule = Utils.DrawMolecule(MolData,texture,draw_type)
    for Bond in Molecule['bond_list']: Fig.add_trace(Bond)
    for Atom in Molecule['atom_list']: Fig.add_trace(Atom)
    Fig.update_layout(Utils.GetLayout(None))
    # MinRange = numpy.min(Molecule['geometry'])
    # MaxRange = numpy.max(Molecule['geometry'])
    # Fig.update_layout(Utils.GetRange(MinRange,MaxRange))
    Sphere = go.Surface(x=x_sphere*radius, y=y_sphere*radius, z=z_sphere*radius, customdata=col_sphere, opacity=alphval,surfacecolor=col_sphere,contours={"x":{"show":True},"y":{"show":True},"z":{"show":True}},colorscale='Turbo')
    Energies = go.Scatter3d(x=x_data,y=y_data,z=z_data,mode='markers',marker=dict(size=5,color=en_data,colorscale='Turbo',opacity=1))
    Contours = go.Scatter3d(x=x_contour,y=y_contour,z=z_contour,mode='markers',marker=dict(size=3,color='black'),hovertext=val_contour)
    Fig.add_trace(Energies)
    if interp == "Sphere" or "Contours": Fig.add_trace(Sphere)
    if interp == "Contours":  Fig.add_trace(Contours)
    return Fig

app.run_server(debug=True, use_reloader=False, host='0.0.0.0', port=8080)

Dash is running on http://0.0.0.0:8080/

Dash is running on http://0.0.0.0:8080/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
